In [1]:
!pip install docling langchain langchain-community langchain-huggingface faiss-cpu

In [2]:
!pip install -qU langchain-ollama



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
from typing import List
from langchain_ollama import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter  # For text chunking
from langchain_huggingface import HuggingFaceEmbeddings  # For text embeddings
from langchain_community.vectorstores import FAISS  # Vector database
from langchain.chains import RetrievalQA  # For question-answering pipeline
from langchain.prompts import PromptTemplate  # For customizing LLM prompts

In [4]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = True

doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
            )
        }
    )

/Users/nadimhammoud/Desktop/Chatbot docling/chatbot/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from docling_core.types.doc import DoclingDocument
def extract_pdf_content(converter: DocumentConverter ,file_path) -> DoclingDocument:
    """
    Extract structured content from PDF using Docling library
    
    Args:
        file_path (str): Path to the PDF file
        
    Returns:
        str: Extracted content in markdown format
    """
    result = converter.convert(file_path)
    return result.document


file_path = "Input-context.pdf"
document = extract_pdf_content(doc_converter,file_path)

In [6]:
def saveMarkdown(result):
    with open("extracted.md", "w",  encoding="utf-8") as mdFile:
        mdFile.write(result)


def save_tables_to_csv(tables, base_filename="table"):
    """
    Save a list of pandas DataFrames to CSV files in a specified directory.

    Args:
        tables (List[pd.DataFrame]): List of pandas DataFrames.
        output_dir (str or Path): Folder where CSVs should be saved.
        base_filename (str): Base name for each CSV file.
    """
    output_dir = "./Tables"
    try:
        os.makedirs(output_dir, exist_ok= False)
    except FileExistsError:
        print("Files already exists will not override!")
        return

    for i, df in enumerate(tables):
        filename = f"{output_dir}/{base_filename}_{i+1}.csv"
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Saved table {i+1} to {filename}")

saveMarkdown(document.export_to_markdown())
tables = [table.export_to_dataframe() for table in document.tables]
save_tables_to_csv(tables)

Files already exists will not override!


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000,  # Maximum chunk size in characters
        chunk_overlap=600,  # Overlap between chunks to maintain context
        is_separator_regex=False
    )

with open("extracted.md", "r",  encoding="utf-8") as mdFile:
    structured_content = mdFile.read()

text_chunks = text_splitter.split_text(structured_content)


In [8]:
from langchain.document_loaders import CSVLoader
folder_path = './Tables'
files = os.listdir(folder_path)
all_documents = []

# Loop through each file and load documents
for csv_file in files:
    loader = CSVLoader(f"{folder_path}/{csv_file}")
    documents = loader.load()
    all_documents.extend(documents)  # Append the loaded documents to the list

In [9]:
def create_vector_store(texts: List[str]) -> FAISS:
    """
    Create and initialize FAISS vector store using text embeddings
    
    Args:
        texts (List[str]): List of text chunks to be embedded
        
    Returns:
        FAISS: Initialized FAISS vector store containing embedded texts
    """
    # Initialize sentence transformer model for text embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L12-v2"
    )
    vector_store = FAISS.from_texts(texts, embeddings)
    return vector_store

vector_store = create_vector_store(text_chunks)
vector_store.add_documents(all_documents)

['c9e2539f-ba36-47ba-9807-0993555ed390',
 'b508c684-9463-41fc-89ed-7958996c53fc',
 '1f11af9f-df56-4277-8eb0-72cf09eeb8e0',
 '86454c03-1d26-449f-9a4d-1af2780ec0fe',
 'd5d8daec-cc65-4ad9-9067-caf2c37a89d0',
 'eae83db1-e01f-43ab-ace1-b04c4548fe8e',
 '6f9e670c-d754-4143-aa2f-8bd0d665967a',
 'a1dd3349-6db8-4de0-ba60-a907aec91118',
 '09d9523a-cdc0-47cf-8fe0-1e461cc37408',
 '5316d1a1-713d-4d65-8003-baa95976ef11',
 '5051e691-bbd1-4c4a-9c5d-b63c5a78ae46',
 '6e07cb32-e730-4f4b-94c1-5890f77a9b62',
 '5d492072-9512-43b4-832d-abdd90614579',
 '73bcd786-591b-48d2-a537-db154a790d71',
 '3e5a44a7-d31a-41b5-9a61-c8606a742c87',
 'd2bc6dda-e5f4-4316-acab-40efe49c444b',
 '9762f46a-3085-403d-8f70-85f498eaaeca',
 '16a9c28c-df8c-496b-abd2-fb4bb49c8bd6',
 '183f7178-c0e7-410b-b611-8648bb0c2911',
 '07098868-e3d4-4e77-bd7a-e57dabb75d8e',
 'bb4c97f8-a1db-47b7-aafa-fa059e8fb3a9',
 'd6ba5df3-b9f0-4b66-bf89-685a146c5970',
 '5cfa306f-3139-48f6-9274-8004df35fd3b',
 '26b37eec-43c9-4711-9ef2-d2dd902337e6',
 '03f9bfbd-e9ca-

In [10]:
from uuid import uuid4
from langchain_core.chat_history import InMemoryChatMessageHistory

session_id = str(uuid4())  # Or a user/session ID from your app
store = {}  # Simple in-memory store; replace with a persistent one for production

def get_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [11]:
from langchain_core.runnables.history import RunnableWithMessageHistory

def get_chain(vector_store):
    """
    Create question-answering chain using LLM and vector store
    
    Args:
        vector_store (FAISS): Vector store containing embedded documents
        
    Returns:
        RetrievalQA: Configured QA chain for answering questions
    """
    # Initialize local LLM using Ollama
    llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0
    )
    
    # Define custom prompt template for better QA responses
    prompt_template = """
        Use the following pieces of context to answer the question at the end.
        Check context very carefully and reference and try to make sense of that before responding.
        If you don't know the answer, just say you don't know.
        Don't try to make up an answer.
        Answer must be as detailed as possible.
        Think step-by-step.
        If a summary table is available, output it.
        Do not include image placeholders in your response.

        Context: {context}
        
        Question: {question}
        
        Answer:"""
    
    PROMPT = PromptTemplate(
        template=prompt_template, input_variables=["context", "question"]
    )

    # Configure QA chain with retrieval and prompt settings
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Combines all retrieved docs into single context
        retriever=vector_store.as_retriever(search_kwargs={"k": 5}),  # Retrieve top 3 relevant chunks
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True,  # Include source documents in response
    )

    qa_chain_with_history = RunnableWithMessageHistory(
        qa_chain,
        get_session_history=get_history,
        input_messages_key="query",  # default RetrievalQA input key
        history_messages_key="history",
    )

    return qa_chain_with_history

chain = get_chain(vector_store)

In [16]:
question = "walk me through configuring the contacts tab when creating a carrier profile"
print(f"\nQuestion: {question}")
response = chain.invoke({"query": question},
                        config={"configurable": {"session_id": session_id}})
print(f"\nAnswer: {response['result']}")


Question: walk me through configuring the contacts tab when creating a carrier profile


Error in RootListenersTracer.on_chain_end callback: KeyError('output')



Answer: To configure the Contacts tab of a Carrier Profile, follow these steps:

1. First, navigate to the Carrier Profiles page and select the Carrier Profile for which you want to add or edit contacts.
2. Click on the "Edit" action next to the Carrier Profile name to access its settings.
3. In the Carrier Profile settings, click on the "Contacts" tab. This will take you to the Contacts tab of the Carrier Profile.

The Contacts tab allows you to store carrier contact information, including individual contact names and emails that can be used for email alerting, rating, and billing purposes.

4. On the Contacts tab, you will see several fields that need to be filled in:
   - Billing Contact: This field determines the name of the billing contact.
   - Billing Email: This field determines the email address of the billing contact. You can add multiple emails by following these steps:
     1. Select the downward arrow of the Email field.
     2. Enter the required email address.
     3. P

In [ ]:
!pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [14]:
!ollama pull llama3.2:latest #run this to download llama3.2 in the environment so you can use it. 2GB

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Error: could not connect to ollama app, is it running?


In [18]:
import gradio as gr
# Define the function to get the response from the LLM
def get_response(question, history):
    # Simulating the call to chain.invoke (make sure to replace this with your actual implementation)
    response = chain.invoke({"query": question},
                            config={"configurable": {"session_id": session_id}})
    return response['result']

# Use gr.Blocks with a Monochrome theme
with gr.Blocks(theme=gr.themes.Soft(), fill_height=True, fill_width=True) as demo:
        # Textbox for the response output
        answer_output = gr.ChatInterface(fn=get_response,title="Vanrise Assistant", type="messages", editable=True, save_history=True)

# Launch the Gradio interface
demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Error in RootListenersTracer.on_chain_end callback: KeyError('output')
Error in RootListenersTracer.on_chain_end callback: KeyError('output')
